<div style="text-align: center; font-weight: bold; font-size: 200%">Projet - Options basket</div><br />

_A faire pour lundi 16 octobre_

In [1]:
import numpy as np
import scipy.stats as sps
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from numpy.random import default_rng, SeedSequence

sq = SeedSequence()
rng = default_rng(sq)

# Options basket

On considère $d \ge 2$ actifs financiers dont la loi à l'instant $T > 0$ est modélisée par une loi log-normale c'est à dire 
$$
    \forall i \in \{1,\dots,d\}, \quad
    S^i_T = S^i_0 \exp\Bigl( \bigl(r-\frac{\sigma_i^2}{2}\bigr) T + \sigma_i \sqrt{T} \tilde G_i \Bigr)
$$
où le vecteur $(\tilde G_1,\dots, \tilde G_d)$ est gaussien centré de matrice de covariance $\Sigma$ et les constantes $r > 0$, $\sigma_i > 0$ sont fixées. Il s'agit d'actifs financiers $(S^i_t)_{t \in [0,T]}$, $1 \le i \le d$, modélisés par un processus de Black-Scholes multidimensionnel. On introduit la matrice $L$ triangulaire inférieure obtenue par la décomposition de Cholesky de la matrice $\Sigma = L L^\top$. 

A l'aide de cette matrice $L$, on définit la fonction $\Phi:\mathbf{R}^d \to \mathbf{R}^d$ telle que 
$$
    (S^1_T, \dots, S^d_T) = \Phi(G_1, \dots, G_d) \quad \text{ou encore} \quad S^i_T = \Phi_i(G_1, \dots, G_d)
$$
où $(G_1, \dots, G_d) \sim \mathcal{N}(0, I_d)$ (l'égalité précédente est à considérer en loi).

## Approximation Monte Carlo

On s'intéresse au prix d'une option européenne (aussi appelé produit dérivé européen) sur le panier de ces $d$ actifs financiers, c'est à dire qu'on veut calculer 
$$
    e^{-rT} \mathbf{E} \bigl[ X \bigr] %\quad \text{avec} \quad g(x) = (x-K)_+ \quad \text{ou} \quad g(x) = (K-x)_+ 
    \quad \text{avec} \quad 
    X = \biggl(\frac{1}{d} \sum_{i=1}^d S^i_T  - K\biggr)_+
$$

In [2]:
from BlackScholes import BlackScholes
from monte_carlo import monte_carlo
from utility import get_vanilla_payoff
from typing import Union, Any
from numpy.typing import NDArray
from numpy import float_

### Parameters initialization

In [3]:
d = 3
vols = np.array([0.2, 0.25, 0.3])
S_0 = np.array([100, 90, 110])
r = 0.05
T = 1.5
K = [80, 90, 100, 110, 120]
flag = 'c'
corr_mat = np.array([
    [1, 0.7, 0.8],
    [0.7, 1, 0.8],
    [0.8, 0.8, 1]
])
print('Minimal eigenvalue:', np.min(np.linalg.eigvals(corr_mat)))

Minimal eigenvalue: 0.16572807176729867


### Model initialization

In [4]:
cov_mat = vols[:, None] * corr_mat * vols[None, :]

model = BlackScholes(
    sigma=cov_mat,
    r=r
)

In [30]:
def MC_basket_price(
        T: Union[float, NDArray[float_]],
        K: Union[float, NDArray[float_]],
        S: Union[float, NDArray[float_]],
        flag: str,
        model: Any, 
        n_sample: int, 
        random_state: np.random.Generator = None
) -> Union[float, NDArray[float_]]:
    """
        Calculates the basket option price in the given model via Monte Carlo simulation.

        Args:
            T: times to maturity.
            K: strikes.
            S: spot prices at t = 0.
            flag: 'c' for calls, 'p' for puts.
            model: model to simulate trajectories. Should support the method 'simulate_trajectory(n_sample, t_grid, init_val, random_state).
            n_sample: number of the trajectories.
            random_state: `np.random.Generator` used for simulation.

        Returns:
            Monte Carlo price,s and confidence interval,s.
    """
    if random_state is None:
        random_state = np.random.default_rng(seed=42)

    S_T_sample = model.simulate_trajectory(
        n_sample=n_sample,
        t_grid=np.array([T]),
        init_val=S,
        random_state=random_state
    ).squeeze() # array of shape (n_sample, dim)
    basket_sample = S_T_sample.mean(axis=1)
    payoff = get_vanilla_payoff(np.array(K)[None, :], flag)
    sample = payoff(np.exp(-r * T) * basket_sample[:, None])
    return monte_carlo(sample, axis=0)

In [33]:
n_sample = 10**7
rng = np.random.default_rng(seed=0xAB0BA)
prices, CI_devs = MC_basket_price(T, K, S_0, flag, model, n_sample, rng)

In [34]:
for strike, price, CI_dev in zip(K, prices, CI_devs):
    print(f'Monte Carlo price of the option with strike {strike}:', np.round(price, 4), "±", np.round(CI_dev, 4))

Monte Carlo price of the option with strike 80: 23.0967 ± 0.0159
Monte Carlo price of the option with strike 90: 16.4536 ± 0.0142
Monte Carlo price of the option with strike 100: 11.3181 ± 0.0123
Monte Carlo price of the option with strike 110: 7.5605 ± 0.0104
Monte Carlo price of the option with strike 120: 4.9324 ± 0.0085


Implémenter un estimateur Monte Carlo pour calculer au centime près le prix pour différentes valeurs de $K$, $K \in \{80, 90, 100, 110, 120\}$.

## Réduction de variance

Implémenter les méthodes de réduction de variance suivantes, et comparer les temps de calcul et la complexité: 
1. Méthode des variables antithétiques en considérant $(W^1_T, \dots, W^d_T)$ et $(-W^1_T, \dots, -W^d_T)$,
2. Utilisation de la variable de contrôle $Y = \bigl(\bar S_0 e^Z  - K\bigr)_+$ avec $\bar S_0 = \frac{1}{d} \sum_{i=1}^d S^i_0$ et $Z = \sum_{i=1}^d a^i_0 \big(\mu_i T + \sigma_i \sqrt{T} \tilde G_i\big)$,
3. Un mélange des 2 méthodes précédentes.

ToDo:

1. Antithetic variables for Brownian motion sampling
2. Add control variable to the method
3. Justify the choice of $\mu, a_0$.
4. Rewrite Monte Carlo simulation as a class.